In [1]:
from docplex.mp.model import Model
m = Model(name='single variable')
x = m.continuous_var(name="x", lb=0)
c1 = m.add_constraint(x >= 2, ctname="const1")
m.set_objective("min", 3*x)
m.print_information()
m.solve()
m.print_solution()

Model: single variable
 - number of variables: 1
   - binary=0, integer=0, continuous=1
 - number of constraints: 1
   - linear=1
 - parameters: defaults
 - objective: minimize
 - problem type is: LP
objective: 6.000
  x=2.000


In [2]:
import numpy as np
import pandas as pd
import random
import unittest
from itertools import product

In [3]:
from docplex.util.status import JobSolveStatus

## Input data

### We define all the input data for the model.

The containers are of four types which are empty (type $20E$) and full (type $20F$) 20-foot containers, and empty (type $20E$) and full (type $40F$) 40-foot containers. We denote the set of container types as 
$\Pi=\{20E,20F,40E,40F\}$ and the set of vehicle types as $M=\{1,\ldots,4\}$.

Vehicle of type 1 can transport at most one 20-foot container,  vehicle of type 2 can transport at most two 20-foot containers, vehicle of type 3 can transport at most one 40-foot container, and vehicle of type 4 can transport at most one 20-foot container and at most one 40-foot container. The mentioned containers can be empty or full. 

In [4]:
# Define the set of container types (Pi) and vehicle types (M)
Pi = set(["20E", "20F", "40E", "40F"])
M = range(1, 5)
print('Pi: ', Pi)
print('M: ', M)

Pi:  {'20F', '40F', '20E', '40E'}
M:  range(1, 5)


In [5]:
# Define the set of container sizes (H)
H = set([20, 40])

In [6]:
# Define the capacity of each vehicle type
Cap = {
    (1, 20): 1,
    (2, 20): 2,
    (3, 40): 1,
    (4, 20): 1,
    (4, 40): 1
}

In [7]:
# the number of full container pickup requests 
f = 2
# the number of empty container pickup requests
p = 2
# the number of empty container dropoff requests 
d = 2
# the number of AFS
s = 2
# data type. 
data_type = 'B'
# the sample number (determines which distances are used)
n = 1

In [8]:
input_init_data = (str(f) + 'f' + str(p)  + 'p' + str(d) + 'd' + str(s) + 's'+ data_type + '/')
common_path = 'data/' + input_init_data
common_path

'data/2f2p2d2sB/'

In [9]:
data_path = common_path + (str(f) + 'f' + str(p)  + 'p' + str(d) + \
                           'd' + str(s) + 's'+ str(n) + 'n' + data_type + '.txt')
data_path

'data/2f2p2d2sB/2f2p2d2s1nB.txt'

In [10]:
names=['NumNodes', "NumPicNodes", "NumTrucks", "NumStations", 
       "NumPicFull", "NumPicEmpty", "NumDelEmpty", "Tmax", "DistanceMatrix"]

In [11]:
df = pd.read_fwf(data_path, 
                 header=None,
                 names=names, skiprows=1)
df.index += 1 
df

,NumNodes,NumPicNodes,NumTrucks,NumStations,NumPicFull,NumPicEmpty,NumDelEmpty,Tmax,DistanceMatrix
1,8,4,4,2,2,2,2,900,1
2,8,4,6,2,2,2,2,900,1
3,8,4,8,2,2,2,2,900,1
4,8,4,4,2,2,2,2,1000,1
5,8,4,6,2,2,2,2,1000,1
6,8,4,8,2,2,2,2,1000,1


In [12]:
NumNodes = df.iloc[0].NumNodes
print('NumNodes:', NumNodes)
NumTrucks = df.iloc[1].NumTrucks
print('NumTrucks:', NumTrucks)
Tmax = df.iloc[0].Tmax
print('Tmax:', Tmax)

NumNodes: 8
NumTrucks: 6
Tmax: 900


In [13]:
distance_matrix_path = common_path + 'DistanceMatrix2_' + ( 'F' + str(f)  + '_P' + str(2) +\
                           '_D_' + str(s) + 'S_No'+ str(n) + '.txt')
distance_matrix_path

'data/2f2p2d2sB/DistanceMatrix2_F2_P2_D_2S_No1.txt'

In [14]:
DistanceMatrix = pd.read_csv(distance_matrix_path, delimiter="\t", header=None,)
print('Distance Matrix:')
DistanceMatrix

Distance Matrix:


,0,1,2,3,4,5,6,7,8,9,10,11
0,0,95,218,158,158,81,72,48,149,86,161,0
1,95,0,123,139,139,18,88,89,139,9,69,95
2,218,124,0,199,199,138,195,205,208,132,65,218
3,158,139,199,0,0,127,86,111,15,138,185,158
4,158,139,199,0,0,127,86,111,15,138,185,158
5,81,18,138,127,127,0,70,71,126,12,87,81
6,72,88,195,86,86,70,0,25,78,81,154,72
7,48,89,205,111,111,71,25,0,101,81,158,48
8,149,139,208,15,15,126,78,101,0,137,190,149
9,86,9,132,138,138,12,81,81,137,0,78,86


In [15]:
time_matrix_path = common_path + 'TimeMatrix2_' + ( 'F' + str(f)  + '_P' + str(2) +\
                           '_D_' + str(s) + 'S_No'+ str(n) + '.txt')
time_matrix_path

'data/2f2p2d2sB/TimeMatrix2_F2_P2_D_2S_No1.txt'

The road network is given by a pair $(N,Q)$, where $N$ and $Q$ are the set of nodes  and arcs, respectively. The node set $N$ represents client locations that supply and demand  containers, a unique departure depot, a unique arrival depot and vehicle refuel locations.

In [16]:
N = DistanceMatrix.shape[0]
print("N:", N)

N: 12


Each node $i\in N$ is associated with a *node time window* $[a_i,b_i]$, $0\le a_i<b_i\le T$, within which operations at this node must be performed.

In [17]:
node_time_window_path = common_path + "variables" + str(NumNodes) + "_N" + str(Tmax) + "_T" \
                        + str(f) + '_P_No' + str(n) + '.txt'
node_time_window_path

'data/2f2p2d2sB/variables8_N900_T2_P_No1.txt'

In [18]:
# service time = 

In [19]:
node_time_window = pd.read_csv(node_time_window_path, delimiter=" ", header=None)
node_time_window

,0,1,2,3,4,5,6,7,8
0,0,10,10,10,10,10,10,10,10.0
1,290,295,296,300,401,402,403,404,NaN
2,350,480,490,500,861,802,802,804,NaN


In [20]:
service_time = node_time_window.iloc[0][1]
service_time
assert service_time == 10

In [21]:
node_time_window.iloc[1:3, 0:-1]

,0,1,2,3,4,5,6,7
1,290,295,296,300,401,402,403,404
2,350,480,490,500,861,802,802,804


In [22]:
a = np.full(N, np.nan)
for i in range(1, NumNodes+1):
    a[i] = node_time_window.iloc[1, i-1]
a[0] = 0
a[NumNodes+1: N] = 0
a = pd.DataFrame(a.reshape(1,-1), columns=range(N))
a

,0,1,2,3,4,5,6,7,8,9,10,11
0,0.0,290.0,295.0,296.0,300.0,401.0,402.0,403.0,404.0,0.0,0.0,0.0


In [23]:
b = np.full(N, np.nan)
for i in range(1, NumNodes+1):
     b[i] = node_time_window.iloc[2, i-1]
b[0] = Tmax
b[NumNodes+1: N] = Tmax
b = pd.DataFrame(b.reshape(1,-1), columns=range(N))
b

,0,1,2,3,4,5,6,7,8,9,10,11
0,900.0,350.0,480.0,490.0,500.0,861.0,802.0,802.0,804.0,900.0,900.0,900.0


There is a *setup time
p*, $0\le 2p\le \min_{j\in N}\{b_j-a_j\}$, which is needed to set up a vehicle for arrival to or departure from any node. Driving ranges of the fully charged vehicles are limited, and they need refueling (recharging) for continuous operation.

In [24]:
# setup time p = 5 minutes
setup_time = 5
print('setup_time:', setup_time)

setup_time: 5


In [25]:
assert (2 * setup_time <= np.min(b - a, axis=1)).all()

Each vehicle $k$ of type $m\in M$ is associated with a *vehicle time window* $[A_k,B_k]$, $0\le A_k<B_k\le T$, a fixed *activation* cost $c^{(a)}_m$, a *unit-drive-time* cost $c^{(t)}_m$,  and a *full-charge driving time range* $f_m$. 

In [26]:
resource_path = common_path + "Resource" + str(NumTrucks) + "_T" + '.txt'
resource_path

'data/2f2p2d2sB/Resource6_T.txt'

In [27]:
resource = pd.read_csv(resource_path, delimiter=" ", header=None)
resource

,0,1
0,1,0
1,2,0
2,0,1
3,1,1
4,2,0
5,1,1


In [28]:
k = []
for i in range(len(resource)):
    if resource.iloc[i, 0] == 1 and resource.iloc[i, 1] == 0:
        k.append(1)
    if resource.iloc[i, 0] == 2 and resource.iloc[i, 1] == 0:
        k.append(2)
    if resource.iloc[i, 0] == 0 and resource.iloc[i, 1] == 1:
        k.append(3)
    if resource.iloc[i, 0] == 1 and resource.iloc[i, 1] == 1:
        k.append(4)
print('k:', k)

k: [1, 2, 3, 4, 2, 4]


In [29]:
m_k = {}
for item in range(len(k)):
    m_k[item+1] = k[item]
    
m_k

{1: 1, 2: 2, 3: 3, 4: 4, 5: 2, 6: 4}

In [30]:
K = list(m_k.keys())
K

[1, 2, 3, 4, 5, 6]

In [31]:
m_k[6]

4

In [32]:
A = np.full(len(k), np.nan)
B = np.full(len(k), np.nan)
for i in range(0, len(k)):
    A[i] = 0
    B[i] = Tmax

A = pd.DataFrame(A.reshape(1, -1), columns=range(1, len(k)+1))
B = pd.DataFrame(B.reshape(1, -1), columns=range(1, len(k)+1))
A

,1,2,3,4,5,6
0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
B

,1,2,3,4,5,6
0,900.0,900.0,900.0,900.0,900.0,900.0


In [34]:
assert np.all(A >= 0) and np.all(A < B) and np.all(B <= Tmax)

In [35]:
assert Tmax <= max(b.max().max(), B.max().max())

In [36]:
fuel_level_path = common_path + "FuelLevel" + str(NumTrucks) + "_T" + '.txt'
fuel_level_path

'data/2f2p2d2sB/FuelLevel6_T.txt'

In [37]:
fuel_level = pd.read_csv(fuel_level_path, delimiter=" ", header=None)
fuel_level

,0,1,2,3,4,5
0,285,285,285,285,285,285


In [38]:
# activation cost. The activation cost is applied once if the respective vehicle goes 
# from the departure depot to any node diﬀerent from the arrival depot.
c_am = np.full(len(M), 10) 
c_am = pd.DataFrame(c_am.reshape(1, -1), columns=M)
c_am

,1,2,3,4
0,10,10,10,10


In [39]:
# unit-drive-time. The unit-drive-time cost is applied for each unit of the driving time.
c_tm = np.full(len(M), 10) 
c_tm = pd.DataFrame(c_tm.reshape(1, -1), columns=M)
c_tm

,1,2,3,4
0,10,10,10,10


In [40]:
vehicle_speed = 50 # km/h
# full-charge driving time range
f_m = np.full(len(M), np.nan) 
for m in range(0, len(M)):
    f_m[m] =  fuel_level[m].values[0] / 0.45  / 50 * 60
    
    
f_m = pd.DataFrame(f_m.reshape(1, -1), columns=M) # in minutes
f_m

,1,2,3,4
0,760.0,760.0,760.0,760.0


*initial drive time capacity* $O^0_k$ at the departure from the departure depot

In [41]:
# Assume that vehicle has the full bak in the departure depot (d_out).
O_0k = {}
for item in K:
    truck_type = m_k[item]
    # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    # O_0k[item] = f_m[truck_type].values[0]
    O_0k[item] = f_m[truck_type].values[0] / 2
    
O_0k

{1: 380.00000000000006,
 2: 380.00000000000006,
 3: 380.00000000000006,
 4: 380.00000000000006,
 5: 380.00000000000006,
 6: 380.00000000000006}

The node set $N$ is partitioned into the following disjoint subsets: a set $DE^{(H)}$  of empty $H$-foot container  demand nodes, a set $SE^{(H)}$  of empty $H$-foot container  supply nodes,  a set $DF^{(H)}$  of full $H$-foot container demand nodes, a set $SF^{(H)}$  of full $H$-foot container  supply nodes, $H\in\{20,40\}$, a set $R$ of 
*refuel* nodes, the departure depot node $d^{out}$ and the arrival depot node $d^{in}$.

In [42]:
# a set of full H-foot container  supply nodes: SF_H(supply = pick up)
# full container pickup requests (FP).
SF_H = DistanceMatrix.columns[1: 1+f]
SF_H

Int64Index([1, 2], dtype='int64')

In [43]:
# a set of empty H-foot container supply nodes: SE_H (supply = pick up)
# empty container pickup requests (EP)
SE_H = DistanceMatrix.columns[1 + len(SF_H): 1+ len(SF_H)+f]
SE_H

Int64Index([3, 4], dtype='int64')

In [44]:
# a set of refuel nodes
R = DistanceMatrix.columns[-(s+1): -1]
R

Int64Index([9, 10], dtype='int64')

In [45]:
# a set of empty H-foot container demand nodes (demand = drop off)
# ED - empty container dropoff requests (in Sezgi article)
tmp = 1 + len(R)
DE_H = DistanceMatrix.columns[-(tmp+d): -tmp]
DE_H

Int64Index([7, 8], dtype='int64')

In [46]:
# a set of full H-foot container demand nodes (demand = drop off)
# full container dropoff requests (FD)
tmp = 1 + len(R) + len(DE_H)
DF_H = DistanceMatrix.columns[-(tmp+p): -tmp]
DF_H

Int64Index([5, 6], dtype='int64')

In [47]:
# the departure depot node
d_out = DistanceMatrix.columns[0]
print('d_out', d_out)

d_out 0


In [48]:
# the arrival depot node
d_in = DistanceMatrix.columns[-1]
print('d_in:', d_in)

d_in: 11


In [49]:
SF = {h: SF_H for h in H}
SE = {h: SE_H for h in H}
DE = {h: DE_H for h in H}
DF = {h: DF_H for h in H}
print('SF: ', SF)
print('SE:', SE)
print('DE:', DE)
print('DF:', DF)

SF:  {40: Int64Index([1, 2], dtype='int64'), 20: Int64Index([1, 2], dtype='int64')}
SE: {40: Int64Index([3, 4], dtype='int64'), 20: Int64Index([3, 4], dtype='int64')}
DE: {40: Int64Index([7, 8], dtype='int64'), 20: Int64Index([7, 8], dtype='int64')}
DF: {40: Int64Index([5, 6], dtype='int64'), 20: Int64Index([5, 6], dtype='int64')}


In [50]:
assert d_in > d_out, "d_in must be greater than d_out"

In [51]:
all_nodes = np.sort(np.concatenate((SF_H, SE_H, DF_H, DE_H , R , [d_out], [d_in])))
all_nodes

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [52]:
assert len(all_nodes) == len(DistanceMatrix.columns), "Length of concatenated array is not equal to DistanceMatrix"
assert all(elem in DistanceMatrix.columns for elem in all_nodes), "Concatenated array does not contain same elements as DistanceMatrix"
assert all(elem in all_nodes for elem in DistanceMatrix.columns), "DistanceMatrix does not contain same elements as concatenated array"

It is convenient to introduce the set $C=\cup_{H\in\{20,40\}}(DE^{(H)}\cup SE^{(H)}\cup DF^{(H)}\cup SF^{(H)})$ of *client nodes*.

In [53]:
C = np.array(list(set().union(*DE.values(), *SE.values(), *DF.values(), *SF.values())))

print("C:", C)

C: [1 2 3 4 5 6 7 8]


We have $Q=\{(i,j)\mid i\in C\cup R\cup\{d^{out}\}, j\in C\cup R\cup\{d^{in}\}{\color{blue}\cup\{d^{out}\}}, i\ne j\}$.

In [54]:
def get_Q(C, R, d_out, d_in):
    return [(i, j) for i in set(C).union(R, {d_out}) for j in set(C).union(R, {d_in}, {d_out}) if i != j]
Q = get_Q(C, R, d_out, d_in)

 Each arc $(i,j)\in Q$ is associated with a *drive time* $t_{ijm}$ of an $m$-type vehicle, $m\in M$, which is needed for such a vehicle to drive from node $i$ to node $j$. For the arc linking the depot nodes, we define $t_{d^{out}d^{in}m}=0$, $m\in M$. We assume without loss of generality that $(i,j)\in Q$ implies $t_{ijm}\le f_m$ for at least one $m\in M$ because otherwise no vehicle can drive along the arc $(i,j)$. 

In [55]:
m_k

{1: 1, 2: 2, 3: 3, 4: 4, 5: 2, 6: 4}

In [56]:
drive_time = DistanceMatrix / vehicle_speed * 60 # in minutes
t = {(i, j, m): drive_time[i][j] for i, j in Q for m in M}
# Check t[d_out][d_in][m] = 0
assert all(t.get(d_out, {}).get(d_in, {}).get(m, 0) == 0 for m in M) 
# Check t[i][j][m] <= f_m for at least one m in M
for i, j in Q:
    assert any(t[(i,j,m)] <= f_m[m][0] for m in M)
    
t

{(0, 1, 1): 114.0,
 (0, 1, 2): 114.0,
 (0, 1, 3): 114.0,
 (0, 1, 4): 114.0,
 (0, 2, 1): 261.6,
 (0, 2, 2): 261.6,
 (0, 2, 3): 261.6,
 (0, 2, 4): 261.6,
 (0, 3, 1): 189.60000000000002,
 (0, 3, 2): 189.60000000000002,
 (0, 3, 3): 189.60000000000002,
 (0, 3, 4): 189.60000000000002,
 (0, 4, 1): 189.60000000000002,
 (0, 4, 2): 189.60000000000002,
 (0, 4, 3): 189.60000000000002,
 (0, 4, 4): 189.60000000000002,
 (0, 5, 1): 97.2,
 (0, 5, 2): 97.2,
 (0, 5, 3): 97.2,
 (0, 5, 4): 97.2,
 (0, 6, 1): 86.39999999999999,
 (0, 6, 2): 86.39999999999999,
 (0, 6, 3): 86.39999999999999,
 (0, 6, 4): 86.39999999999999,
 (0, 7, 1): 57.599999999999994,
 (0, 7, 2): 57.599999999999994,
 (0, 7, 3): 57.599999999999994,
 (0, 7, 4): 57.599999999999994,
 (0, 8, 1): 178.8,
 (0, 8, 2): 178.8,
 (0, 8, 3): 178.8,
 (0, 8, 4): 178.8,
 (0, 9, 1): 103.2,
 (0, 9, 2): 103.2,
 (0, 9, 3): 103.2,
 (0, 9, 4): 103.2,
 (0, 10, 1): 193.20000000000002,
 (0, 10, 2): 193.20000000000002,
 (0, 10, 3): 193.20000000000002,
 (0, 10, 4): 193.

 Each refuel node $i\in R$ is associated with the minimal refuel time $\delta^{\min}$ and the $m$-type vehicle *it refuel speeds* $e_{mi}$, $e_{mi}\ge f_m/(b_i-a_i-2p)$, $m\in M$. The refuel speed is measured in driving capacity time units per refuel time unit. For example, $e_{mi}=30$ means that 1 minute of refuel time adds 30 drive time minutes to the vehicle.

In [57]:
delta_min = 5 # minutes
print('delta_min:', delta_min)

delta_min: 5


In [58]:
e_mi = pd.DataFrame(columns=R, index=M)
for m in M:
    for i in R:
        e_mi.loc[m, i] = f_m.loc[:,m][0] / (0.1 * fuel_level[0][0])
e_mi

,9,10
1,26.666667,26.666667
2,26.666667,26.666667
3,26.666667,26.666667
4,26.666667,26.666667


In [59]:
demand_path = common_path + "demand" + str(f) + "_F" + str(p) + '_P' + str(d) + '_D.txt'
demand_path

'data/2f2p2d2sB/demand2_F2_P2_D.txt'

In [60]:
# columns: 0 - full 20foot, 1 - empty 20foot, 2 - full 40foot, 3 - empty 40foot
# 1 - pick up / -1 - drop off
demand = pd.read_csv(demand_path, delimiter="\t", header=None)
demand.index +=1
demand

,0,1,2,3
1,1,0,0,0
2,1,0,0,0
3,0,1,0,0
4,0,1,0,0
5,-1,0,0,0
6,-1,0,0,0
7,0,-1,0,0
8,0,-1,0,0


In [61]:
demand_explain = demand

# Rename the columns
demand_explain = demand_explain.rename(columns={0: 'full 20', 1: 'empty 20', 2: 'full 40', 3: 'empty 40'})

# Rename the rows
demand_explain = demand_explain.rename(index={1: '1: (SF)', 2: '2: (SF)', 3: '3: (SE)', 4: '4: (SE)', 5: '5: (DF)', 6: '6: (DF)', 7: '7: (DE)', 8: '8: (DE)'})
demand_explain

,full 20,empty 20,full 40,empty 40
1: (SF),1,0,0,0
2: (SF),1,0,0,0
3: (SE),0,1,0,0
4: (SE),0,1,0,0
5: (DF),-1,0,0,0
6: (DF),-1,0,0,0
7: (DE),0,-1,0,0
8: (DE),0,-1,0,0


Each node $i\in DE^{(H)}$ is associated with a demand quantity $d^{(H)}_i$ and each node $i\in SE^{(H)}$ is associated with a supply quantity $s^{(H)}_i$ of empty $H$-foot containers, $H\in\{20,40\}$.

In [62]:
demand_explain

,full 20,empty 20,full 40,empty 40
1: (SF),1,0,0,0
2: (SF),1,0,0,0
3: (SE),0,1,0,0
4: (SE),0,1,0,0
5: (DF),-1,0,0,0
6: (DF),-1,0,0,0
7: (DE),0,-1,0,0
8: (DE),0,-1,0,0


In [63]:
f_ij = {}
for h in H:
    f_ij[h] = {}
    for item in range(len(SF[h])):
        if h == 20:
            tmp_sf = demand.loc[SF[h][item]][0]
            tmp_df = demand.loc[DF[h][item]][0]
            assert tmp_sf == -tmp_df or (tmp_sf == 0 and tmp_df == 0)
            f_ij[h][(SF[h][item], DF[h][item])] = abs(tmp_df)
        elif h == 40:
            tmp_sf = demand.loc[SF[h][item]][2]
            tmp_df = demand.loc[DF[h][item]][2]
            assert tmp_sf == tmp_df
            f_ij[h][(SF[h][item], DF[h][item])] = abs(tmp_df)
            
f_ij

{40: {(1, 5): 0, (2, 6): 0}, 20: {(1, 5): 1, (2, 6): 1}}

In [64]:
d_Hi = {}
for h in H:
    d_Hi[h] = {}
    for i in DE[h]:
        if h == 20:
            # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
            # d_Hi[h][i] = abs(demand.loc[i][[1]].values[0])
            d_Hi[h][i] = 0
        if h == 40:
            d_Hi[h][i] = abs(demand.loc[i][[3]].values[0])
s_Hi = {}
for h in H:
    s_Hi[h] = {}
    for i in SE[h]:
        if h == 20:
            s_Hi[h][i] = abs(demand.loc[i][[1]].values[0])
        if h == 40:
            s_Hi[h][i] = abs(demand.loc[i][[3]].values[0])

print('d_Hi:', d_Hi)
print('s_Hi:', s_Hi)

d_Hi: {40: {7: 0, 8: 0}, 20: {7: 0, 8: 0}}
s_Hi: {40: {3: 0, 4: 0}, 20: {3: 1, 4: 1}}


In [65]:
d_Hi[20]

{7: 0, 8: 0}

In [66]:
# d_Hi = {}
# for h in H:
#     for i in DE[h]:
#         if h == 20:
#             d_Hi[(i, h)] = abs(demand.loc[i][[1]].values[0])
#         if h == 40:
#             d_Hi[(i, h)] = abs(demand.loc[i][[3]].values[0])
# s_Hi = {}
# for h in H:
#     for i in SE[h]:
#         if h == 20:
#             s_Hi[(i, h)] = abs(demand.loc[i][[1]].values[0])
#         if h == 40:
#             s_Hi[(i, h)] = abs(demand.loc[i][[3]].values[0])

# print('d_Hi:', d_Hi)
# print('s_Hi:', s_Hi)

There is a one-to-one correspondence between nodes of the sets  $SF^{(H)}$ and  $DF^{(H)}$ such that each *origin node* $i\in SF^{(H)}$ corresponds to a unique *destination node* $j\in DF^{(H)}$ and vice versa. Define the sets of *origin-destination arcs* as $F^{(H)}=\{(i,j)\in Q\mid i\in SF^{(H)}, j\in DF^{(H)}\}$, $H\in\{20,40\}$. Each arc $(i,j)\in F^{(H)}$ is associated with a *requested exact flow* $f^{(H)}_{ij}$ of full $H$-foot containers between $i$ and $j$, $H\in\{20,40\}$. It implies that the node $i\in SF^{(H)}$ possesses $f^{(H)}_{ij}$ full $H$-foot containers and the node $j\in DF^{(H)}$ demands these containers if $(i,j)\in F^{(H)}$, $H\in\{20,40\}$.

In [67]:
F = {}
for h in H:
    assert len(SF[h]) == len(DF[h]), "SF and DF have different lengths for h={}".format(h)
    F_h = []
    for item in range(0, len(SF[h])):
        F_h.append((SF[h][item], DF[h][item]))
    F[h] = F_h
print('F:', F)

F: {40: [(1, 5), (2, 6)], 20: [(1, 5), (2, 6)]}


In [68]:
f_ij = {}
for h in H:
    f_ij[h] = {}
    for item in range(len(SF[h])):
        if h == 20:
            tmp_sf = demand.loc[SF[h][item]][0]
            tmp_df = demand.loc[DF[h][item]][0]
            assert tmp_sf == -tmp_df or (tmp_sf == 0 and tmp_df == 0)
            f_ij[h][(SF[h][item], DF[h][item])] = abs(tmp_df)
        elif h == 40:
            tmp_sf = demand.loc[SF[h][item]][2]
            tmp_df = demand.loc[DF[h][item]][2]
            assert tmp_sf == tmp_df
            f_ij[h][(SF[h][item], DF[h][item])] = abs(tmp_df)

print('f_ij:', f_ij)

f_ij: {40: {(1, 5): 0, (2, 6): 0}, 20: {(1, 5): 1, (2, 6): 1}}


In [69]:
# f_ij = {}
# for h in H:
#     f_ij[h] = []
#     for item in range(len(SF[h])):
#         if h == 20:
#             tmp_sf = demand.loc[SF[h][item]][0]
#             tmp_df = demand.loc[DF[h][item]][0]
#             assert tmp_sf == -tmp_df or (tmp_sf == 0 and tmp_df == 0)
#             f_ij[h].append(( {(SF[h][item], DF[h][item]): abs(tmp_df)} ))
#         elif h == 40:
#             tmp_sf = demand.loc[SF[h][item]][2]
#             tmp_df = demand.loc[DF[h][item]][2]
#             assert tmp_sf == tmp_df
#             f_ij[h].append(({(SF[h][item], DF[h][item]): abs(tmp_df)}))

# print('f_ij:', f_ij)

Each non-{\color{blue}arrival-}depot node $i\in R\cup C{\color{blue}\cup\{d^{out}\}}$ is equipped with $l_i$ parallel identical {\it service lines} (refuel lines, unloading-loading lines), denoted as $(i,l)$, $l\in L_i=\{1,\ldots,l_i\}$, such that no line can service more than one vehicle at a time.  We associate each service line $(i,l)$, $i\in C\cup R{\color{blue}\cup\{d^{out}\}}$, $l\in L_i$, with a  set $V_i=\{1,2,\ldots,v^{\max}_i\}$ of possible visits of this line by all vehicles.
The visits are numbered in the increasing order of the vehicle arrival times. Thus, $v\in V_i$ can be viewed as a position in the visiting sequence of a line $(i,l)$ by all vehicles. The same vehicle can visit any line any number of times within the available positions. Not all $v^{\max}_i$ visits can be realized in a vehicle timetable.  If a visit is realized then we call it as real, otherwise, we call it as artificial.
For modelling purposes, it is convenient to introduce a single artificial line and a single visit position <font color="blue">for the arrival depot node:$L_{d^{in}}=V_{d^{in}}=\{1\}$. This single visit position can be occupied by all vehicles simultaneously. It is assumed that there is an initial supply of $S^{(HE)}_l$ of empty $H$-foot containers in line $l$ of the departure depot node $d^{out}$, $H\in\{20,40\}$ </font>.

In [70]:
RCd_out = np.hstack((R,C, d_out))
li, v_max_i = 2, 2 
l_i = {}
v_i =  {}
for i in RCd_out:
    l_i[i] = li # l_i, v_max_i = 2, 2 
    v_i[i] = v_max_i
    
print('l_i', l_i)
print('v_i', v_i)

l_i {9: 2, 10: 2, 1: 2, 2: 2, 3: 2, 4: 2, 5: 2, 6: 2, 7: 2, 8: 2, 0: 2}
v_i {9: 2, 10: 2, 1: 2, 2: 2, 3: 2, 4: 2, 5: 2, 6: 2, 7: 2, 8: 2, 0: 2}


In [71]:
L = {}
V = {}
for i in RCd_out:
    L[i] = set(range(1, l_i[i] + 1))
    V[i] = set(range(1, v_i[i] + 1))
    
L[d_in], V[d_in] = {1}, {1}
print('L:', L)
print('V:', V)

L: {9: {1, 2}, 10: {1, 2}, 1: {1, 2}, 2: {1, 2}, 3: {1, 2}, 4: {1, 2}, 5: {1, 2}, 6: {1, 2}, 7: {1, 2}, 8: {1, 2}, 0: {1, 2}, 11: {1}}
V: {9: {1, 2}, 10: {1, 2}, 1: {1, 2}, 2: {1, 2}, 3: {1, 2}, 4: {1, 2}, 5: {1, 2}, 6: {1, 2}, 7: {1, 2}, 8: {1, 2}, 0: {1, 2}, 11: {1}}


Assume that an $m$-type vehicle $k$ unloads $r$ containers, $r\in\{0,1,2\}$, and loads $h$ containers, $h\in\{0,1,2\}$,  at visit $v$ of <font color="blue"> a non-arrival-depot </font> line $(i,l)$. The respective events happen in the following sequence:  arrival, arrival setup, $r$ unloading operations and $h$ loading operations in any order, departure setup and departure. 

In [72]:
r = {0,1,2}
h = {0,1,2}

##### MILP model

It is convenient to introduce the following additional notation.
* $Pred_i\subset N$ -- 	set of immediate predecessor nodes of node $i\in N$;
* $Succ_i\subset N$ -- set of immediate successor nodes of node $i\in N$;	
* $E=\{((v,i,l),(u,j,q))\mid v\in V_i,\ l\in L_i,\ u\in V_j,\ q\in L_j,\ (i,j)\in Q\}$.


In [73]:
Pred = {}
Succ = {}
for item in all_nodes:
    if item == d_in:
        Pred[item] = np.delete(all_nodes, item)
        Succ[item] = []
    if item == d_out or item in np.hstack((R,C)):
        Pred[item] = np.delete(all_nodes, [item, d_in])
        Succ[item] = np.delete(all_nodes, item)


In [74]:
E = []
E = [(v,i,l, u,j,q) for (i, j) in Q for v in V[i] for l in L[i] for u in V[j] for q in L[j]]

In [75]:
# Create CPLEX model
model = Model(name='VRP')

$x^0_{kvil}$  -- binary variable equal to 1 if vehicle $k$ occupies position $v$ in the visiting sequence of the line $(i,l)$, $k\in K$, $v\in V_i$, $i\in N$, $l\in L_i$

In [76]:
x0 = {}
for i in all_nodes:
    for l in L[i]:
        for k in K:
            for v in V[i]:
                x0[k,v,i,l] = model.binary_var(name=f'x0_{k}_{v}_{i}_{l}')

$x_{kvilujq}$ -- binary variable equal to 1 if vehicle $k$ drives over the arc $((v,i,l),(u,j,q))\in E$

In [77]:
x = {}
for k in K:
    for item in E:
        v, i, l, u, j, q = item
        x[k, v, i, l, u, j, q] = model.binary_var(name=f'x_{k}_{v}_{i}_{l}_{u}_{j}_{q}')

$Pick^{(\pi)}_{kvil}$ -- number of $\pi$-type containers that vehicle $k$ picks at visit $v$ of the line $(i,l)$ , $k\in K$, $v\in V_i$, $i\in N$, $l\in L_i$, $\pi\in\Pi$

In [78]:
Pick = {}
for pi in Pi:
    for k in K:
        for i in all_nodes:
            for l in L[i]:
                for v in V[i]:
                    name = f'Pick_{pi}_{k}_{v}_{i}_{l}'
                    Pick[pi, k, v, i, l] = model.integer_var(name=name)

$Drop^{(\pi)}_{kvil}$ -- number of $\pi$-type containers that vehicle $k$ drops at visit $v$ of the line $(i,l)$, $k\in K$, $v\in V_i$, $i\in N$, $l\in L_i$, $\pi\in\Pi$

In [79]:
Drop = {}
for pi in Pi:
    for k in K:
        for i in all_nodes:
            for l in L[i]:
                for v in V[i]:
                    name = f'Drop_{pi}_{k}_{v}_{i}_{l}'
                    Drop[pi, k, v, i, l] = model.integer_var(name=name)

$Load^{(\pi)}_{kvil}$ -- number of $\pi$-type containers that vehicle $k$ carries immediately before visit $v$ of the line $(i,l)$, $k\in K$, $v\in V_i$, $i\in N$, $l\in L_i$, $\pi\in\Pi$

In [80]:
Load = {}
for pi in Pi:
    for k in K:
        for i in all_nodes:
            for l in L[i]:
                for v in V[i]:
                    name = f'Load_{pi}_{k}_{v}_{i}_{l}'
                    Load[pi, k, v, i, l] = model.integer_var(name=name)

$X_{kvil}$ -- time of arrival of vehicle $k$ to the line $(i,l)$ for visit $v$, $k\in K$, $v\in V_i$, $i\in N$, $l\in L_i$

In [81]:
X = {}
for k in K:
    for i in all_nodes:
        for l in L[i]:
            for v in V[i]:
                name = 'X' + '_' + str(k) + '_' + str(v) + '_' + str(i) + '_' + str(l)
                X[k, v, i, l] = model.continuous_var(name=name)

$Y_{kvil}$ -- time of departure of vehicle $k$ from the line $(i,l)$ after visit $v$, $k\in K$, $v\in V_i$, $i\in N$, $l\in L_i$

In [82]:
Y = {}
for k in K:
    for i in all_nodes:
        for l in L[i]:
            for v in V[i]:
                name = f'Y_{k}_{v}_{i}_{l}'
                Y[k, v, i, l] = model.continuous_var(name=name)

$I_{kvil}$  --  remaining drive time capacity of vehicle $k$ at its arrival to the line $(i,l)$ for visit~$v$, $k\in K$, $v\in V_i$, $i\in N$, $l\in L_i$

In [83]:
I = {}
for k in K:
    for i in all_nodes:
        for l in L[i]:
            for v in V[i]:
                name = f'I_{k}_{v}_{i}_{l}'
                I[k, v, i, l] = model.continuous_var(name=name)

$O_{kvil}$ --  remaining drive time capacity of vehicle $k$ at its departure from the line $(i,l)$ after visit $v$, $k\in K$, $v\in V_i$, $i\in N$, $l\in L_i$. Note that $I_{kvil}=O_{kvil}$ for $i\not\in R$

In [84]:
O = {}
for k in K:
    for i in all_nodes:
        for l in L[i]:
            for v in V[i]:
                name = f'O_{k}_{v}_{i}_{l}'
                O[k, v, i, l] = model.continuous_var(name=name)

$\delta_{kvil}$ --  refuel time of vehicle $k$ at visit $v$ of the refuel line $(i,l)$, $k\in K$, $v\in V_i$, $i\in R$, $l\in L_i$

In [85]:
delta = {}
for k in K:
    for i in R:
        for l in L[i]:
            for v in V[i]:
                name = f'delta_{k}_{v}_{i}_{l}'
                delta[k, v, i, l] = model.continuous_var(name=name)

<font color="blue">$\sigma_{vl}^{(H)}$ --  supply of $H$-foot empty containers at line $l$ of the departure depot $d^{out}$ in the end of visit $v$ of this line, $v\in V_{d^{out}}$, $\sigma_{0l}^{(H)}=S^{(HE)}_l$, $l\in L_{d^{out}}$, $H\in\{20,40\}$. </font>

In [86]:
sum_d_Hi = {}
for key in d_Hi.keys():
    # initialize a counter variable
    count = 0
    # loop over the values of the inner dictionary
    for value in d_Hi[key].values():
        # add the value to the counter
        count += value
    # store the result for the current key
    sum_d_Hi[key] = count
    
print(sum_d_Hi)

{40: 0, 20: 0}


In [87]:
# Define the decision variables
sigma_Hvl = {}
for v in [0] + [v for v in V[d_out]]:
    for l in L[d_out]:
        for h in H:
            sigma_Hvl[h,v,l] = model.continuous_var(name=f'sigma_{h}_{v}_{l}')

In [88]:
# Define the constraints
for l in L[d_out]:
    for h in H:
        model.add_constraint(sigma_Hvl[h, 0, l] == sum_d_Hi[h])  # Constraint 1
    for v in V[d_out]:
        model.add_constraint(sigma_Hvl[20, v, l] >= 0)     # Constraint 2
        model.add_constraint(sigma_Hvl[20, v, l] >= 0)     # Constraint 3

Our MILP model can be written as follows. Denote a pair of expressions $v\in  V_i$ and $l\in L_i$ as $\forall (v,l)$, a pair of expressions $u\in  V_j$ and $q\in L_j$ as $\forall (u,q)$, and  a pair of expressions $w\in  V_h$ and $g\in L_h$ as $\forall (w,g)$.

$$ \min \sum_{k\in K}\sum_{((v,i,l),(u,j,q))\in E}c^{(t)}_{m(k)}t_{ij,m(k)}x_{kvilujq}+
\sum_{k\in K,\forall (v,l),i\in Succ_{d^{out}}\backslash\{d^{in}\}}c^{(a)}_{m(k)}x^0_{kvil} $$

In [89]:
obj = 0
for k in K:
    for (v, i, l, u, j, q) in E:
        obj += c_tm[m_k[k]].values[0] * t[(i,j,m_k[k])] * x[(k, v, i, l, u, j, q)]
    for i in Succ[d_out]:
        for l in L[i]:
            for v in V[i]:
                if i != d_in:
                    obj += c_am[m_k[k]].values[0] * x0[(k, v, i, l)]
model.minimize(obj)

#### subject to
predefined variable values and  <font color="blue"> no relocation for all but empty containers at </font>d^{out}

<font color="blue"> 
$$ Load^{(\pi)}_{k1d^{in}1}???=Drop^{(\pi)}_{k1d^{in}1}=Pick^{(\pi)}_{k1d^{in}1}=0,k\! \in\!  K,\ \pi\in \Pi, \label{EqLout1} (2)$$
</font>

In [90]:
for pi in Pi:
    for k in K:
        model.add_constraint(Load[(pi, k, 1, d_in, 1)] == Drop[(pi, k, 1, d_in, 1)], ctname="EqLout1_Eq2_1")
        model.add_constraint(Drop[(pi, k, 1, d_in, 1)] == Pick[(pi, k, 1, d_in, 1)], ctname="EqLout1_Eq2_2")


<span style="color:blue">
$$Drop^{(\pi)}_{k1d^{out}1}=Pick^{(\pi)}_{k1d^{out}1}=0,\ k\in K,\ \pi\in\{DF^{(20)},DF^{(40)}\}  \label{EqLout2} (3)$$
</span>

In [91]:
# DF(20) and DF(40) are the same nodes. This model (to test the Sezgi model) 
# does not divide the nodes into two types of containers: 20 foot and 40 foot. 
for pi in ['20F', '40F']:
    for k in K:
        model.add_constraint(Load[pi, k, 1, d_out, 1] == 0, ctname="EqLout2_Eq3_1")
        model.add_constraint(Drop[pi, k, 1, d_out, 1] == 0, ctname="EqLout2_Eq3_2")
        model.add_constraint(Pick[pi, k, 1, d_out, 1] == 0, ctname="EqLout2_Eq3_3")

$$ Drop^{(HE)}_{kvil}=0, k\! \in\!  K, \forall (v,l), i\!\not\!\in\! DE^{(H)},\ H\in\{20,40\},   \label{EqDrop1}  (4)$$ 

In [92]:
for HE in ['20E', '40E']:
    for k in K:
        for i in np.delete(all_nodes, DE_H):
            for l in L[i]:
                for v in V[i]:
                    model.add_constraint(Drop[HE, k, v, i, l] == 0, ctname="EqDrop1_Eq4")

$$ Drop^{(HF)}_{kvil}=0, k\! \in\!  K, \forall (v,l), i\!\not\!\in\! DF^{(H)},  \ H\in\{20,40\},   \label{EqDrop2} (5)$$

In [93]:
for HE in ['20F', '40F']:
    for k in K:
        for i in np.delete(all_nodes, DF_H):
            for l in L[i]:
                for v in V[i]:
                    model.add_constraint(Drop[HE, k, v, i, l] == 0, ctname="EqDrop2_Eq5")

$$
Pick^{(HE)}_{kvil}=0, k\! \in\!  K, \forall (v,l), i\!\not\!\in\! SE^{(H)}, \ H\in\{20,40\},   \label{EqPick1} (6)
$$

In [94]:
for HE in ['20E', '40E']:
    for k in K:
        for i in np.delete(all_nodes, SE_H):
            for l in L[i]:
                for v in V[i]:
                    model.add_constraint(Drop[HE, k, v, i, l] == 0, ctname="EqPick1_Eq6")

$$
Pick^{(HF)}_{kvil}=0, k\! \in\!  K, \forall (v,l), i\!\not\!\in\! SF^{(H)}, \ H\in\{20,40\},    \label{EqPick2} (7)
$$

In [95]:
for HE in ['20F', '40F']:
    for k in K:
        for i in np.delete(all_nodes, SF_H):
            for l in L[i]:
                for v in V[i]:
                    model.add_constraint(Drop[HE, k, v, i, l] == 0, ctname="EqPick2_Eq7")

$$ x^0_{k1d^{out}1}=1, k\! \in\!  K, \label{Eqx0Dout}  (8) $$

In [96]:
for k in K:
    model.add_constraint(x0[k, 1, d_out,1] == 1, ctname="Eqx0Dout_Eq8")

$$ x^0_{k1d^{in}1}=1, k\! \in\!  K, (9) $$

In [97]:
for k in K:
    model.add_constraint(x0[k, 1, d_in,1] == 1, ctname="Eqx0Din_Eq9")

<span style="color:blue">
$$
{\color{blue}O_{k1d^{out}1}=O^0_k, k\! \in\!  K,} \label{EqDriveout}  (10)
$$
</span>

In [98]:
for k in K:
    model.add_constraint(O[k,1,d_out,1] == O_0k[m_k[k]], ctname=f"Driveout_Eq10")

###### Feasible load of vehicles

$$
Load^{(\pi)}_{kujq}=(Load^{(\pi)}_{kvil}\! -\! Drop^{(\pi)}_{kvil}\! +\! Pick^{(\pi)}_{kvil})x_{kvilujq},\mbox{ --- quadratic ---}  k\! \in\!  K, \forall (v,l), i\! \in\!  C{\color{blue}\cup\{d^{out}\}}, \forall (u,q), j\! \in\!  C{\color{blue}\cup\{d^{out}\}}, \pi\! \in\!  \Pi, 
 \nonumber \label{EqFeaLge20} (11)
$$ 

###### Linearization

$$
Load^{(\pi)}_{kujq}\le 2x_{kvilujq}, \label{Elineariz}  (45)
$$

In [99]:
for k in K:
    for pi in Pi:
        for i, j in Q:
            if i in np.hstack((C, d_out)) and j in np.hstack((C, d_out)):
                for (v, l) in product(V[i], L[i]):
                    for (u, q) in product(V[j], L[j]): 
                        model.add_constraint(Load[pi,k,u,j,q] <= 2 * x[k,v,i,l,u,j,q], 
                                        ctname="Elineariz_Eq45")

$$
 Load^{(\pi)}_{kvil}\! -\! Drop^{(\pi)}_{kvil}\! +\! Pick^{(\pi)}_{kvil}\le Load^{(\pi)}_{kujq} +2(1-x_{kvilujq}),\label{Elineariz2} (46)
$$

In [100]:
for k in K:
    for pi in Pi:
        for i, j in Q:
            if i in np.hstack((C, d_out)) and j in np.hstack((C, d_out)):
                for (v, l) in product(V[i], L[i]):
                    for (u, q) in product(V[j], L[j]):
                        left = Load[pi,k,v,i,l] - Drop[pi,k,v,i,l] + Pick[pi,k,v,i,l] 
                        right = Load[pi,k,u,j,q] + 2 * (1 - x[k,v,i,l,u,j,q])
                        model.add_constraint(left <= right, 
                                        ctname="Elineariz2_Eq46")

$$
Load^{(\pi)}_{kujq}\le Load^{(\pi)}_{kvil}\! -\! Drop^{(\pi)}_{kvil}\! +\! Pick^{(\pi)}_{kvil}.    \label{Elineariz3} (47)
$$

In [101]:
for k in K:
    for pi in Pi:
        for i, j in Q:
            if i in np.hstack((C, d_out)) and j in np.hstack((C, d_out)):
                for (v, l) in product(V[i], L[i]):
                    for (u, q) in product(V[j], L[j]):
                        left = Load[pi,k,u,j,q]
                        right = Load[pi,k,v,i,l] - Drop[pi,k,v,i,l] + Pick[pi,k,v,i,l]
                        model.add_constraint(left <= right, 
                                        ctname="Elineariz3_Eq47")

$$
Drop^{(\pi)}_{kvil}\le Load^{(\pi)}_{kvil},k\! \in\!  K, \forall (v,l), i\! \in\!  C{\color{blue}\cup\{d^{out}\}}, \pi\! \in\!  \Pi,\label{EqFeaVisit1} (12)
$$

In [102]:
for k in K:
    for i in np.hstack((C, d_out)):
        for (v, l) in product(V[i], L[i]):
            for pi in Pi:
                model.add_constraint(Drop[pi,k,v,i,l] <= Load[pi,k,v,i,l],
                             ctname="EqFeaVisit1_Eq12")

$$
Load^{(HE)}_{kvil}\! +\! Pick^{(HE)}_{kvil}\! +\! Load^{(HF)}_{kvil}\! +\! Pick^{(HF)}_{kvil}\! \le\!  Cap^{(H)}_{m(k)}x^0_{kvil}, \label{EqFeaLge2H}\\
k\! \in\!  K, \forall (v,l), i\! \in\!  C{\color{blue}\cup\{d^{out}\}}, H\in\{20,40\},(13)
$$

In [103]:
for k in K:
    for i in np.hstack((C, d_out)):
        for (v, l) in product(V[i], L[i]):
            for h in [20, 40]:
                HE = str(h) + 'E'
                HF = str(h) + 'F'
                left = Load[HE,k,v,i,l] + Pick[HE, k,v,i,l] + Load[HF,k,v,i,l] + Pick[HF, k,v,i,l]
                right = Cap.get((m_k[k], h), 0) * x0[k,v,i,l]
                model.add_constraint(left <= right, ctname="EqFeaLge2H_Eq13")

###### feasible drive time capacity of vehicles

$$ \nonumber  \\
I_{kujq}=O_{kvil}-t_{ijm(k)}x_{kvilujq},k\! \in\!  K, \forall ((v,i,l), (j,u,q))\!\in\! E, \label{EqDrive0}(14) $$

In [104]:
for k in K:
    for (v, i, l, u, j, q) in E:
        model.add_constraint(I[k, u, j, q] == 
                        O[k, v, i, l] - t[i, j, m_k[k]]*x[k, v, i, l, u, j, q], ctname="EqDrive0_Eq14")


$$
I_{kvil}=O_{kvil}, k\in K,\forall (v,l), i\in C{\color{blue}\cup\{d^{out}\}},  \label{EqDriveEqual} (15)
$$

In [105]:
for k in K:
    for i in np.hstack((C, d_out)):
        for (v, l) in product(V[i], L[i]):
            model.add_constraint(I[k,v,i,l] == O[k, v, i, l], ctname="EqDriveEqual_Eq15")

$$ I_{kvil}+e_{m(k),i}\delta_{kvil}\le f_{m(k)}x^0_{kvil}, k\! \in\!  K, \forall (v,l),  i\! \in\!  R,  (16)  $$

In [106]:
for k in K:
    for i in R:
        for (v, l) in product(V[i], L[i]):
            model.add_constraint(I[k, v, i, l] + e_mi.loc[m_k[k],i] * delta[k, v, i, l] <= f_m[m_k[k]].values[0] * x0[k, v, i, l], 
                            ctname=f"EqDrivex0_Eq16")

$$ \delta^{\min} x^0_{kvil}\le\delta_{kvil}, k\! \in\!  K, \forall (v,l),  i\! \in\!  R,  \label{EqDrivedelt2} (17)$$

In [107]:
for k in K:
    for i in R:
        for (v, l) in product(V[i], L[i]):
            model.add_constraint(delta_min * x0[k,v,i,l] <= delta[k, v, i, l], ctname=f"EqDrivedelt2_Eq17")

$$ O_{kvil}=I_{kvil}+e_{m(k),i}\delta_{kvil},\ k\in K,\forall (v,l), i\in R,\label{EqDrive2} (18)$$

In [108]:
for k in K:
    for i in R:
        for (v, l) in product(V[i], L[i]):
            model.add_constraint(O[k, v, i, l] == I[k, v, i, l] + e_mi.loc[m_k[k],i] * delta[k, v, i, l], 
                            ctname=f"EqDrive2_Eq18")

###### linking departure and arrival times

$$ X_{kujq}\ge Y_{kvil}+t_{ijm(k)}x_{kvilujq},k\! \in\!  K, \forall ((v,i,l), (j,u,q))\!\in\! E,  \label{EqLinkXY} (19)$$

In [109]:
for k in K:
    for (v, i, l, u, j, q) in E:
        model.add_constraint(X[k, u, j, q] >= Y[k, v, i, l] + t[(i,j,m_k[k])] * x[k,v,i,l,u,j,q], 
                           ctname=f"EqLinkXY_Eq19")

###### node time window feasibility

$$ a_i+2p+\delta_{kvil}\le b_i+T(1-x^0_{kvil}), k\! \in\!  K, \forall (v,l), i\! \in\! R,  \label{EqYXR1}  (20)$$

In [110]:
for k in K:
    for i in R:
        for (v, l) in product(V[i], L[i]):
            left = a[i].values[0] + 2 * setup_time + delta[k,v,i,l]
            right = b[i].values[0] + Tmax * (1 - x0[k,v,i,l])
            model.add_constraint(left <= right, ctname=f"EqYXR1_Eq20")

$$ a_i+2p+\delta_{kvil}\le Y_{kvil}+T(1-x^0_{kvil}), k\! \in\!  K, \forall (v,l), i\! \in\! R, \label{EqYXR2}  (21)$$

In [111]:
for k in K:
    for i in R:
        for (v, l) in product(V[i], L[i]):
            left = a[i].values[0] + 2 * setup_time + delta[k,v,i,l]
            right = Y[k,v,i,l] + Tmax * (1 - x0[k,v,i,l])
            model.add_constraint(left <= right, ctname=f"EqYXR2_Eq21")

$$ X_{kvil}+2p+\delta_{kvil}\le b_i+T(1-x^0_{kvil}), k\! \in\!  K, \forall (v,l), i\! \in\! R,  \label{EqYXR3} (22)$$

In [112]:
for k in K:
    for i in R:
        for (v, l) in product(V[i], L[i]):
            left = X[k,v,i,l] + 2 * setup_time + delta[k,v,i,l]
            right = b[i].values[0] + Tmax * (1 - x0[k,v,i,l])
            model.add_constraint(left <= right, ctname=f"EqYXR3_Eq22")

$$ X_{kvil}+2p+\delta_{kvil}\le Y_{kvil}+T(1-x^0_{kvil}), k\! \in\!  K, \forall (v,l), i\! \in\! R, \label{EqYXR4} (23)$$

In [113]:
for k in K:
    for i in R:
        for (v, l) in product(V[i], L[i]):
            left = X[k,v,i,l] + 2 * setup_time + delta[k,v,i,l]
            right = Y[k,v,i,l] + Tmax * (1 - x0[k,v,i,l])
            model.add_constraint(left <= right, ctname=f"EqYXR4_Eq23")

$$
a_i\!+\! 2p\!+o\!\Big(\sum_{\pi\!\in\!\Pi}\sum_{k\!\in\! K} Drop^{(\pi)}_{kvil}\!+\!\sum_{\pi\!\in\!\Pi}\sum_{k\!\in\! K} Pick^{(\pi)}_{kvil}\Big)\!\le\! b_i\!+\!T(1\!-\! x^0_{kvil}),  \label{EqYXR1c} \\
k\! \in\!  K, \forall (v,l), i\! \in\! C{\color{blue}\cup\{d^{out}\}}, (24)
$$

In [114]:
for i in np.hstack((C, d_out)):
    for (v, l) in product(V[i], L[i]):
        lhs = a[i].values[0] + 2*setup_time + sum(Drop[(pi, k, v, i, l)] + Pick[(pi, k, v, i, l)] for pi in Pi for k in K)
        rhs = b[i].values[0] + Tmax*(1 - x0[k, v, i, l])
        model.add_constraint(lhs <= rhs, ctname="EqYXR1c_Eq24")

$$ 
a_i\!+ 2p\! +\! o\Big(\sum_{\pi\!\in\!\Pi}\sum_{k\!\in\! K} Drop^{(\pi)}_{kvil}\! +\!\sum_{\pi\!\in\!\Pi}\sum_{k\!\in\! K} Pick^{(\pi)}_{kvil}\Big)\!\le\! Y_{kvil}\!+\!T(1\!-\! x^0_{kvil}),\label{EqYXR2c}  \\
k\! \in\!  K, \forall (v,l), i\! \in\! C{\color{blue}\cup\{d^{out}\}},
(25)
$$

In [115]:
for i in np.hstack((C, d_out)):
    for (v, l) in product(V[i], L[i]):
        lhs = a[i].values[0] + 2*setup_time + sum(Drop[(pi, k, v, i, l)] + Pick[(pi, k, v, i, l)] for pi in Pi for k in K)
        rhs = Y[k,v,i,l] + Tmax*(1 - x0[k, v, i, l])
        model.add_constraint(lhs <= rhs, ctname="EqYXR2c_Eq25")

$$ X_{kvil}\!+ 2p\! +\! o\Big(\sum_{\pi\!\in\!\Pi}\sum_{k\!\in\! K} Drop^{(\pi)}_{kvil}\! +\! \sum_{\pi\!\in\!\Pi}\sum_{k\!\in\! K} Pick^{(\pi)}_{kvil}\Big)\!\le\! b_i\!+\! T(1\!-\! x^0_{kvil}),  \label{EqYXR3c}\\
k\! \in\!  K, \forall (v,l), i\! \in\! C{\color{blue}\cup\{d^{out}\}}, (26)$$

In [116]:
for i in np.hstack((C, d_out)):
    for (v, l) in product(V[i], L[i]):
        lhs = X[k,v,i,l] + 2*setup_time + sum(Drop[(pi, k, v, i, l)] + Pick[(pi, k, v, i, l)] for pi in Pi for k in K)
        rhs = b[i].values[0] + Tmax*(1 - x0[k, v, i, l])
        model.add_constraint(lhs <= rhs, ctname="EqYXR3c_Eq26")

$$ X_{kvil}\!+\! 2p\!  +\! o\Big(\sum_{\pi\!\in\!\Pi}\sum_{k\!\in\! K} Drop^{(\pi)}_{kvil}\! +\! \sum_{\pi\!\in\!\Pi}\sum_{k\!\in\! K} Pick^{(\pi)}_{kvil}\Big)\!\le\! Y_{kvil}\!+\! T(1\! -\! x^0_{kvil}), \label{EqYXR4c} \\
k\! \in\!  K, \forall (v,l), i\! \in\! C{\color{blue}\cup\{d^{out}\}} (27)$$

In [117]:
for i in np.hstack((C, d_out)):
    for (v, l) in product(V[i], L[i]):
        lhs = X[k,v,i,l] + 2*setup_time + sum(Drop[(pi, k, v, i, l)] + Pick[(pi, k, v, i, l)] for pi in Pi for k in K)
        rhs = Y[k,v,i,l] + Tmax*(1 - x0[k, v, i, l])
        model.add_constraint(lhs <= rhs, ctname="EqYXR4c_Eq27")

###### vehicle time window feasibility

$$ A_k\le Y_{k1d^{out}1}, k\! \in\!  K,  \label{EqVehTWout} (28)$$

In [118]:
dictA = A.to_dict(orient='records')[0]
for k in K:
    model.add_constraint(dictA[k] <= Y[k,1,d_out,1], ctname="EqVehTWout_Eq28")

$$ X_{k1d^{out}1}\le B_k, k\! \in\!  K \label{EqVehTWin}  (29)$$

In [119]:
for k in K:
    model.add_constraint(Y[k,1,d_out,1]<= B[k].values[0], ctname="EqVehTWin_Eq29")

###### non-intersecting visits of any <span style="color:blue"> non-arrival-</span> depot line

$$ \sum_{k\in K} Y_{kvil}\le \sum_{k\in K} X_{k,v+1,il},\ v=1,\ldots,v_i-1,\ l\in L_i,\ i\in C\cup R{\color{blue} \cup\{d^{out}\}}, \label{EqYXR42} (30)$$

In [120]:
for i in np.hstack((C, R, d_out)):
    for l in L[i]:
        for v in range(1, v_i[i]):
            lhs = sum(Y[k,v,i,l] for k in K)
            rhs = sum(X[k,v+1,i,l] for k in K)
            model.add_constraint(lhs <= rhs, ctname="EqYXR42_Eq30")

###### satisfying full container flow

$$ \sum_{\forall (v,l)}\sum_{k\in K}Pick^{(HF)}_{kvil}=\sum_{\forall (u,q)}\sum_{k\in K}Drop^{(HF)}_{kujq}=f^{(H)}_{ij},\ (i,j)\in F^{(H)}, H\in\{20,40\},\label{EqFlowH} (31)$$

In [121]:
for h in H:
    for i,j in F[h]:
        HF = str(h) + 'F'
        constr1 = sum(Pick[HF,k,v,i,l] for k in K for v in V[i] for l in L[i])
        constr2 = sum(Drop[HF,k,u,j,q] for k in K for u in V[j] for q in L[j])
        model.add_constraint(constr1 == constr2)
        model.add_constraint(constr1 == f_ij[20].get((i, j)), ctname="EqFlow20_Eq31")

###### satisfying empty container demand

$$ \sum_{\forall (v,l)}\sum_{k\in K}Drop^{(HE)}_{kvil}=d^{(H)}_i,\ i\in DE^{(H)},\ H\in\{20,40\}, \label{EqDH}   (32)$$

In [122]:
for h in H:
    for i in DE[h]:
        HE = str(h) + 'E'
        constr1 = sum(Drop[HE,k,v,i,l] for k in K for v in V[i] for l in L[i])
        constr2 = d_Hi[h].get(i)
        model.add_constraint(constr1 == constr2, ctname="EqDH_Eq32")

###### satisfying empty container supply

$$ \sum_{\forall (v,l)}\sum_{k\in K}Pick^{(HE)}_{kvil}\le s^{(H)}_i,\ i\in SE^{(H)},\ H\in\{20,40\}, \label{EqSH} (33)$$

In [123]:
for h in H:
    for i in SE[h]:
        HE =  str(h) + 'E'
        constr1 = sum(Pick[HE,k,v,i,l] for k in K for v in V[i] for l in L[i])
        constr2 = s_Hi[h].get(i)
        model.add_constraint(constr1 == constr2, ctname="EqSH_Eq33")


######  <span style="color:blue"> empty container supply in the end of visit v of line l  of departure depot </span>

<span style="color:blue">
$$
{\color{blue}\sigma_{vd^{out}l}^{(H)}\! =\! \sigma_{v-1,d^{out}l}^{(H)}\! +\! \sum_{k\in K}Drop^{(HE)}_{kvd^{out}l}\! -\! \sum_{k\in K}Pick^{(HE)}_{kvd^{out}l},v\!\in\! V_{d^{out}},l\in L_{d^{out}},H\in\{20,40\},} \label{EqDefd_outH} 
(34) $$
</span>

In [124]:
sigma_Hvl

{(40, 0, 1): docplex.mp.Var(type=C,name='sigma_40_0_1'),
 (20, 0, 1): docplex.mp.Var(type=C,name='sigma_20_0_1'),
 (40, 0, 2): docplex.mp.Var(type=C,name='sigma_40_0_2'),
 (20, 0, 2): docplex.mp.Var(type=C,name='sigma_20_0_2'),
 (40, 1, 1): docplex.mp.Var(type=C,name='sigma_40_1_1'),
 (20, 1, 1): docplex.mp.Var(type=C,name='sigma_20_1_1'),
 (40, 1, 2): docplex.mp.Var(type=C,name='sigma_40_1_2'),
 (20, 1, 2): docplex.mp.Var(type=C,name='sigma_20_1_2'),
 (40, 2, 1): docplex.mp.Var(type=C,name='sigma_40_2_1'),
 (20, 2, 1): docplex.mp.Var(type=C,name='sigma_20_2_1'),
 (40, 2, 2): docplex.mp.Var(type=C,name='sigma_40_2_2'),
 (20, 2, 2): docplex.mp.Var(type=C,name='sigma_20_2_2')}

In [125]:
for v in V[d_out]:
    for l in L[d_out]:
         for h in H:
                HE = str(h) + 'E'
                constr1 = sigma_Hvl[h,v,l]
                constr2 = sigma_Hvl[h,v-1,l] + sum(Drop[HE,k,v,d_out,l] for k in K) - sum(Pick[HE,k,v,d_out,l] for k in K)
                model.add_constraint(constr1 == constr2, ctname="EqDefd_outH_Eq34")

######  <span style="color:blue"> satisfying empty container supply and relocation at departure depot </span>

<span style="color:blue">
$$
{\color{blue}\sum_{k\in K}Pick^{(HE)}_{kvd^{out}l}\le \sigma^{(H)}_{vl},\ v\in V_{{d^{out}}},l\in L_{d^{out}},\ H\in\{20,40\},} \label{EqSd_outH} (35)
$$
 </span>

In [126]:
for v in V[d_out]:
    for l in L[d_out]:
         for h in H:
                HE = str(h) + 'E'
                constr1 = sum(Pick[HE, k,v,d_out,l] for k in K) 
                constr2 = sigma_Hvl[h,v,l]
                model.add_constraint(constr1 <= constr2, ctname="EqSd_outH_Eq35")

###### all vehicles leave departure depot

$$ \sum_{i\in Succ_{d^{out}},\forall (v,l)}x_{k1d^{out}1vil}=1,\ k\in K,  \label{Eq4}  (36)$$

In [127]:
for k in K:
    model.add_constraint(
            sum(
                x[k,1,d_out,1,v,i,l]
                for i in Succ[d_out]
                for v in V[i]
                for l in L[i]
            ) == 1,
            ctname="Eq4_Eq36"
        )

###### all vehicles return to arrival depot

$$ \sum_{i\in Pred_{d^{in}},\forall (v,l)} x_{kvil1d^{in}1}=1,\ k\in K,   \label{Eq6} (37)$$

In [128]:
for k in K:
    model.add_constraint(
            sum(
                x[k,v,i,l,1,d_in,1]
                for i in Pred[d_in]
                for v in V[i]
                for l in L[i]
            ) == 1,
            ctname="Eq6_Eq37"
        )

###### at most one vehicle arrives to any non-depot line for the same visit

$$ \sum_{k\in K}x^0_{kvil}\le 1,\forall (v,l), i\in C\cup R, \label{EqIn5}  (38)$$

In [129]:
for i in R.union(C):
    for v in V[i]:
        for l in L[i]:
            model.add_constraint(
                sum(
                    x0[k, v, i, l]
                    for k in K
                ) <= 1,
                ctname="EqIn5_Eq38"
            )

###### real visits are consecutively indexed

$$ \sum_{k\in K}x^0_{kvil}\ge \sum_{k\in K}x^0_{k,v+1,il},\ v=1,\ldots,v_i-1,\ l\in L_i,\ i\in C\cup R, \label{EqIn52} (39) $$ 

In [130]:
for i in R.union(C):
    for v in range(1, v_i[i]):
        for l in L[i]:
            model.add_constraint(
                sum(
                    x0[k, v, i, l]
                    for k in K
                ) >= sum(
                    x0[k, v+1, i, l]
                    for k in K
                ),
                ctname="EqIn52_Eq39"
            )

###### if vehicle comes to a non-depot line for a visit then it must leave it

$$  \sum_{i\in Pred_h} \sum_{\forall (v,l)}x_{kvilwhg}=\sum_{j\in Succ_h} \sum_{\forall (u,q)}x_{kwhgujq}=x^0_{kwhg},\  k\in K, \forall (w,g),  h\in C\cup R,  \label{Eq5}  (40)$$

In [131]:
for k in K:
    for h in R.union(C):
        for w in V[h]:
            for g in L[h]:
                model.add_constraint(
                    sum(
                        x[k,v,i,l,w,h,g]
                        for i in Pred[h]
                        for v in V[i]
                        for l in L[i]
                    ) == x0[k,w,h,g],
                    ctname="Eq5_Eq40_1"
                )
                model.add_constraint(
                    sum(
                        x[k,w,h,g,u,j,q]
                        for j in Succ[h]
                        for u in V[j]
                        for q in L[j]
                    ) == x0[k,w,h,g],
                    ctname="Eq5_Eq40_2"
                )


###### Domains of variables

$$ x^0_{kvil}\!\in\!  \{0,1\},\  x_{kvilujq}\!\in\! \{0,1\},\  k\in K,\forall ((v,i,l), (i,u,q))\!\in\! E,  \label{EqDx} (41)$$

Note that the constraints $$x^0_{kvil}\!\in\! \{0,1\}$$ and $$x_{kvilujq}\!\in\! \{0,1\}$$ are binary constraints, which means that they can only take values 0 or 1. Therefore, we need to add lower bounds of 0 and upper bounds of 1 to these variables.

In [132]:
for k in K:
    for (v,i,l, u,j,q) in E:
        model.add_constraint(x0[k, v, i, l] <= 1, ctname="EqDx_Eq41_1")
        model.add_constraint(x0[k, v, i, l] >= 0, ctname="EqDx_Eq41_2")
        model.add_constraint(x[k, v, i, l, u, j, q] <= 1, ctname="EqDx_Eq41_3")
        model.add_constraint(x[k, v, i, l, u, j, q] >= 0, ctname="EqDx_Eq41_4")

$$ \max\{a_i,A_k\}\! \le\!  X_{kvil}\! \le\!  \min\{b_i,B_k\},\max\{a_i,A_k\}\! \le\!  Y_{kvil}\! \le\!  \min\{b_i,B_k\},  k\!\in\! K,\forall (v,l), i\!\in\! N \label{EqDX} (42)$$

In [133]:
for i in all_nodes:
    for v in V[i]:
        for l in L[i]:
            for k in K:
                model.add_constraint(X[k, v, i, l] >= max(a[i].values[0], A[k].values[0]), ctname="EqDX_Eq42_1")
                model.add_constraint(X[k, v, i, l] <= min(b[i].values[0], B[k].values[0]), ctname="EqDX_Eq42_2")
                model.add_constraint(Y[k, v, i, l] >= max(a[i].values[0], A[k].values[0]), ctname="EqDX_Eq42_3")
                model.add_constraint(Y[k, v, i, l] <= min(b[i].values[0], B[k].values[0]), ctname="EqDX_Eq42_4")

$$ 0\le \delta_{kvil}\le f_{m(k)},\ \ k\in K,\forall (v,l),  i\in R,  \label{EqDIdelta}  (43)$$

In [134]:
for k in K:
    for i in R:
        for v in V[i]:
            for l in L[i]:
                upper_bound = f_m[m_k[k]].values[0]  # extract the upper bound value from the series
                model.add_constraint(0 <= delta[k, v, i, l], ctname="EqDIdelta_Eq43_1")
                model.add_constraint(delta[k, v, i, l] <= upper_bound, ctname="EqDIdelta_Eq43_2")

$$ 0\le I_{kvil}\le f_{m(k)},\ \ \  0\le O_{kvil}\le f_{m(k)},\ \ k\!\in\! K,\forall (v,l),  i\!\in\! N.  \label{EqDI}  (44)$$

In [135]:
for i in all_nodes:
    for v in V[i]:
        for l in L[i]:
            for k in K:
                model.add_constraint(0 <= I[k, v, i, l], ctname="EqDI_Eq44_1")
                model.add_constraint(I[k, v, i, l] <= f_m[m_k[k]].values[0], ctname="EqDI_Eq44_2")
                model.add_constraint(0 <= O[k, v, i, l], ctname="EqDI_Eq44_3")
                model.add_constraint(O[k, v, i, l] <= f_m[m_k[k]].values[0], ctname="EqDI_Eq44_4")

In [136]:
def get_conflict(mdl):
    from docplex.mp.conflict_refiner import ConflictRefiner
    print("conflict")
    print()

    cr = ConflictRefiner()
    conflicts = cr.refine_conflict(mdl)
    print(mdl.get_solve_status())
    print('conflicts', conflicts)
    for conflict in conflicts:
        print('FOR in conflicts started')
        st = conflict.status
        ct = conflict.element
        label = conflict.name
        label_type = type(conflict.element)

        # Print conflict information in console
        print("Conflict involving constraint: %s" % label)
        print(" \tfor: %s" % ct)
        print(" \tstatus: %s" % st)
        if st == "infeasible":
            conflict_details = conflict.get_details()
            for constr in conflict_details:
                print(" \t\t%s" % constr)



In [137]:
solution = model.solve(log_output=True)

Version identifier: 12.10.0.0 | 2019-11-26 | 843d4de
CPXPARAM_Read_DataCheck                          1
Row 'EqDrive0_Eq14' infeasible, all entries at implied bounds.
Presolve time = 0.06 sec. (34.87 ticks)

Root node processing (before b&c):
  Real time             =    0.07 sec. (46.79 ticks)
Parallel b&c, 12 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.07 sec. (46.79 ticks)


In [138]:
status = model.get_solve_status()
if status == JobSolveStatus.INFEASIBLE_SOLUTION:
    get_conflict(model)

conflict

JobSolveStatus.INFEASIBLE_SOLUTION
conflicts <docplex.mp.conflict_refiner.ConflictRefinerResult object at 0x11f305cd0>
FOR in conflicts started
Conflict involving constraint: Driveout_Eq10
 	for: Driveout_Eq10: O_1_1_0_1 == 380.00000000000006
 	status: ConflictStatus.Member
FOR in conflicts started
Conflict involving constraint: EqDrive0_Eq14
 	for: EqDrive0_Eq14: I_1_1_9_1 == -103.200x_1_1_0_1_1_9_1+O_1_1_0_1
 	status: ConflictStatus.Member
FOR in conflicts started
Conflict involving constraint: EqDrive0_Eq14
 	for: EqDrive0_Eq14: I_1_1_0_1 == -114x_1_1_1_1_1_0_1+O_1_1_1_1
 	status: ConflictStatus.Member
FOR in conflicts started
Conflict involving constraint: EqDrive0_Eq14
 	for: EqDrive0_Eq14: I_1_1_9_1 == -10.800x_1_1_1_1_1_9_1+O_1_1_1_1
 	status: ConflictStatus.Member
FOR in conflicts started
Conflict involving constraint: EqDrive0_Eq14
 	for: EqDrive0_Eq14: I_1_1_0_1 == -103.200x_1_1_9_1_1_0_1+O_1_1_9_1
 	status: ConflictStatus.Member
FOR in conflicts started
Conflict in